In [11]:
import json
from pathlib import Path

# 문자 길이 기준 최대 청크 크기
MAX_CHARS = 500  # 필요하면 800, 1000 등으로 조정

def load_json(path: str):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def save_json(data, path: str):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def build_full_text(record: dict) -> str:
    """
    한 레코드에서 청킹 전에 사용할 전체 텍스트를 만든다.
    - title
    - content
    - checklist 항목들
    - terms (용어 정리)
    를 하나의 문자열로 합친다.
    """
    parts = []

    # 제목
    title = record.get("title")
    if title:
        parts.append(title)

    # 내용
    content = record.get("content")
    if content:
        parts.append(content)

    # 체크리스트
    checklist = record.get("checklist") or []
    if checklist:
        checklist_text = "\n".join(f"- {item}" for item in checklist)
        parts.append("체크리스트:\n" + checklist_text)

    # 용어 정리
    terms = record.get("terms") or {}
    if terms:
        term_lines = []
        for term, desc in terms.items():
            term_lines.append(f"{term}: {desc}")
        terms_text = "\n".join(term_lines)
        parts.append("용어 정리:\n" + terms_text)

    # 빈 줄로 구분해서 합치기
    full_text = "\n\n".join(parts)
    return full_text.strip()

def split_text_by_length(text: str, max_chars: int):
    """
    문장 단위로 최대 max_chars 길이에 맞춰 잘게 나눈다.
    - 너무 단순하게 공백/마침표 기준으로 자르면 어색해질 수 있어
      그래서 여기서는 '.', '?', '!', '다.' 정도를 기준으로 분리하는 예시를 사용.
    """
    if len(text) <= max_chars:
        return [text]

    # 간단한 문장 분리 (필요하면 더 고도화 가능)
    # 한국어에선 '다.'를 많이 쓰니 이것도 포함
    import re
    # 종결 패턴: "다.", ".", "?", "!"
    sentences = re.split(r'(?<=[\.?!])\s+', text)

    chunks = []
    current = ""

    for sent in sentences:
        if not sent:
            continue
        # 문장을 다시 붙일 때 공백 하나 추가
        candidate = (current + " " + sent).strip() if current else sent.strip()

        if len(candidate) <= max_chars:
            current = candidate
        else:
            # 기존 current를 하나의 청크로 확정
            if current:
                chunks.append(current)
            # 현재 문장이 max_chars보다 큰 경우 → 강제로 잘라서 넣기
            if len(sent) > max_chars:
                start = 0
                while start < len(sent):
                    end = start + max_chars
                    chunks.append(sent[start:end])
                    start = end
                current = ""
            else:
                current = sent.strip()

    if current:
        chunks.append(current)

    return chunks

def chunk_record(record: dict, max_chars: int):
    """
    단일 레코드를 여러 청크로 나누고,
    각 청크에 메타데이터를 붙여 반환한다.
    """
    full_text = build_full_text(record)
    text_chunks = split_text_by_length(full_text, max_chars)

    base_metadata = {
        "source_id": record.get("id"),
        "platform": record.get("platform"),
        "service": record.get("service"),
        "scenario": record.get("scenario"),
        "category": record.get("category"),
        "title": record.get("title"),
        "source_type": record.get("source_type"),
        "updated_at": record.get("updated_at"),
    }

    chunk_results = []
    for idx, chunk_text in enumerate(text_chunks, start=1):
        chunk_id = f"{record.get('id')}_chunk_{idx}"
        chunk_results.append(
            {
                "id": chunk_id,
                "text": chunk_text,
                "metadata": base_metadata,
            }
        )
    return chunk_results

def main():
    input_path = Path("./data/online_shoppingmal.json")
    output_path = Path("./data/online_shoppingmal.json".replace(".json", "_chunked.json"))

    records = load_json(str(input_path))

    all_chunks = []
    for record in records:
        record_chunks = chunk_record(record, MAX_CHARS)
        all_chunks.extend(record_chunks)

    save_json(all_chunks, str(output_path))
    print(f"총 청크 개수: {len(all_chunks)}")
    print(f"저장 위치: {output_path}")

if __name__ == "__main__":
    main()

총 청크 개수: 12
저장 위치: data\online_shoppingmal_chunked.json
